In [1]:
import numpy as np

# Definicja funkcji sigmoidalnej
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Funkcja próbkowania Gibbsa
def gibbs_sampling(v, W, h_bias):
    h_prob = sigmoid(np.dot(v, W) + h_bias)
    h = (np.random.rand(*h_prob.shape) < h_prob).astype(float)
    v_prob = sigmoid(np.dot(h, W.T))
    v_sample = (np.random.rand(*v_prob.shape) < v_prob).astype(float)
    return v_sample

# Funkcja Contrastive Divergence
def contrastive_divergence(v0, W, h_bias, k=1, learning_rate=0.1):
    v = v0
    v_bias = np.zeros_like(v0)
    h0_prob = sigmoid(np.dot(v, W) + h_bias)
    h0 = (np.random.rand(*h0_prob.shape) < h0_prob).astype(float)

    v_k = v
    for _ in range(k):
        v_k = gibbs_sampling(v_k, W, 0)  

    h_k_prob = sigmoid(np.dot(v_k, W) + h_bias)
    h_k = (np.random.rand(*h_k_prob.shape) < h_k_prob).astype(float)

   
    dW = np.outer(v0, h0) - np.outer(v_k, h_k)
    dv_bias = v0 - v_k
    dh_bias = h0 - h_k

    W += learning_rate * dW
    h_bias += learning_rate * dh_bias

    return h_k  

# Przykładowe dane treningowe (binarne wartości)
data = np.array([[1, 0, 1, 0],
                 [1, 1, 0, 0],
                 [0, 0, 1, 1]])

# Inicjalizacja RBM
num_visible = data.shape[1]
num_hidden = 2
W = np.random.rand(num_visible, num_hidden)
h_bias = np.zeros(num_hidden)

# Uczenie RBM przy użyciu Contrastive Divergence
epochs = 1000
learning_rate = 0.1
for epoch in range(epochs):
    for v0 in data:
        contrastive_divergence(v0, W, h_bias, k=1, learning_rate=learning_rate)

# Nowe dane, dla których chcemy przewidzieć wartości "wizualne"
new_data = np.array([[1, 0, 0, 1],
                     [0, 1, 1, 0]])

# Generowanie reprezentacji ukrytej dla nowych danych
hidden_representation = []
for v0 in new_data:
    h_representation = contrastive_divergence(v0, W, h_bias, k=1, learning_rate=learning_rate)
    hidden_representation.append(h_representation)

hidden_representation = np.array(hidden_representation)

# Wyświetlenie wyników
print("Nowe dane:")
print(new_data)
print("\nReprezentacja ukryta:")
print(hidden_representation)


Nowe dane:
[[1 0 0 1]
 [0 1 1 0]]

Reprezentacja ukryta:
[[0. 1.]
 [0. 1.]]
